# Process Data

In [ ]:
import numpy as np
import pandas as pd
import struct
import array
import plotly.express as px
import plotly.graph_objects as go
import math
import time

In [ ]:
import onnx

In [ ]:
import onnxruntime

In [ ]:
from MotionPreprocessing import MotionProcessor, read_csv_style_data, read_csv_data, ReadBinary,FrameRange

In [ ]:
pd.options.display.max_columns = None

In [ ]:
NUM_PHASE_CHANNEL = 5

In [ ]:
IN_FEATURES = 475
NUM_SAMPLES = 28349

In [ ]:
mp = MotionProcessor()
dfInputDataExp = mp.input_preprocessing()

In [ ]:
dfInputDataExp

In [ ]:
fig = go.Figure()

# Extract columns that start with 'PhaseSpace-'
phasespace_columns = [col for col in dfInputDataExp.columns if col.startswith('PhaseSpace-')]
# Filter even and odd indices based on numerical part of column names
even_indices = [col for col in phasespace_columns if int(col.split('-')[-1]) % 2 == 0]
odd_indices = [col for col in phasespace_columns if int(col.split('-')[-1]) % 2 != 0]

colors = ['red', 'green', 'blue', 'orange', 'purple']

for step in range(100):
    row = dfInputDataExp.iloc[step]
    
    # Extract values based on even and odd indices
    p_y = row[even_indices].values.flatten()
    p_x = row[odd_indices].values.flatten()

    # Assign colors to each point, cycling through the colors list every 5 points
    marker_colors = [colors[i % len(colors)] for i in range(len(p_x))]

    fig.add_trace(
            go.Scatter(
                visible=False,
                mode='markers',
                name="𝜈 = " + str(step),
                x=p_x,
                y=p_y,
                marker=dict(color=marker_colors)  # Set marker colors
            ))
    
fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)
fig.update_scenes(aspectmode='cube',aspectratio=dict(x=1, y=1))

fig.update_layout(
    title = "2D Phase Vector (12 Frame Window)",
    xaxis=dict(range=[-5, 5]), 
    yaxis=dict(range=[-5, 5]),  
    autosize=False,
    width=500,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig.show()

In [ ]:
fig.write_html("file_phase.html", include_plotlyjs=False)

### Build Output Features

In [ ]:
OUT_FEATURES = 436

In [ ]:
dfOutDataExp = mp.output_preprocessing()

In [ ]:
dfOutDataExp

In [ ]:
sequence = dfOutDataExp.xs(key=3, level="SeqId")

In [ ]:
sequence

In [ ]:
px.scatter(sequence, x="delta_x", y="delta_y")

In [ ]:
def translation_matrix(dx, dy, dz):
    """Create a 4x4 translation matrix."""
    return np.array([
        [1, 0, 0, dx],
        [0, 1, 0, dy],
        [0, 0, 1, dz],
        [0, 0, 0, 1]
    ])

def rotation_matrix_y(theta):
    """Create a 4x4 rotation matrix around the Y-axis."""
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return np.array([
        [cos_theta, 0, sin_theta, 0],
        [0, 1, 0, 0],
        [-sin_theta, 0, cos_theta, 0],
        [0, 0, 0, 1]
    ])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
p_x =[]
p_y =[]

root = np.eye(4)
origin = [0.0,0.0,0.0,1.0]

position = [0,0,0,1]
for index, row in sequence.iterrows():
    #, y = rotate_vector(row.delta_x, row.delta_y, angle)
    t_mat = translation_matrix(row.delta_x, 0.0, row.delta_y)
    rot_mat = rotation_matrix_y(-row.delta_angle)
    
    tout =  rot_mat @ t_mat 
    
    root =  root @ tout

    pos = root @ origin
    
    p_x.append(pos[0])
    p_y.append(pos[2])

# Create the plot with Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=p_x, y=p_y, mode='markers', name='Position'))

# Set the aspect ratio and axis ranges


fig.update_layout(

    xaxis=dict(range=[-800, 100]), 
    yaxis=dict(range=[-200, 700]),  
    autosize=False,
    width=600,
    height=700,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)
#fig.update_xaxes(constrain='domain')  
#fig.update_yaxes(scaleanchor= 'x')
#fig['layout'].update(scene=dict(aspectmode="data"))

fig.show()

## Export Data

In [ ]:
mp.export_data()

In [ ]:
test = mp.OutputData.iloc[200]

In [ ]:
fig = go.Figure()

# Extract columns that start with 'PhaseSpace-'
#phasespace_columns = [col for col in dfInputDataExp.columns if col.startswith('root_pos')]
phasespace_columns = [col for col in sequence.columns if 'root_pos' in col]
# Filter even and odd indices based on numerical part of column names
even_indices = [col for col in phasespace_columns if "y" in col]
odd_indices = [col for col in phasespace_columns if "x" in col]

for step in range(300):
    #row = dfInputDataExp.iloc[step]
    row = sequence.iloc[step]
    
    # Extract values based on even and odd indices
    p_y = row[even_indices].values.flatten()
    p_x = row[odd_indices].values.flatten()

    fig.add_trace(
            go.Scatter(
                visible=False,
                mode='markers',
                name="𝜈 = " + str(step),
                x=p_x,
                y=p_y))
    
fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)
fig.update_scenes(aspectmode='cube',aspectratio=dict(x=1, y=1))

fig.update_layout(
    title = "2D Phase Vector (12 Frame Window)",
    xaxis=dict(range=[-300, 300]), 
    yaxis=dict(range=[-300, 300]),  
    autosize=False,
    width=500,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig.show()

## Inference

In [ ]:
model_path = r"C:\DEV\AI4Animation\AI4Animation\SIGGRAPH_2022\PyTorch\GNN\Training\144.onnx"

In [ ]:
#onnx setup
session = onnxruntime.InferenceSession(model_path)
inputs = session.get_inputs()
outputs = session.get_outputs()

In [ ]:
# Check network input and output dimensions
[print(f"{n.name}, {n.shape}") for n in inputs]
[print(f"{n.name}, {n.shape}") for n in outputs]

In [ ]:
# Load previously created input binary
test_in = ReadBinary(r"C:\DEV\AnimHost\python\data\Input.bin", 600, 605)
df_inference_input = pd.DataFrame(test_in, columns=mp.get_raw_input_columns())
x_in = df_inference_input.iloc[500].to_numpy(dtype=np.float32).reshape(1,-1)

In [ ]:
# Load previously created output binary
y_groundtruth =  ReadBinary(r"C:\DEV\AnimHost\python\data\Output.bin", 600, 576)
df_groundtruth = pd.DataFrame(y_groundtruth, columns=mp.get_raw_output_columns())

In [ ]:
# Time before code execution
start_time = time.time()


result = session.run(["Y","W"], {"X" : x_in})

# Time after code execution
end_time = time.time()
delta_time = end_time - start_time
print(f"Code execution took {delta_time:.4f} seconds.")

In [ ]:
mp.get_raw_output_columns()

In [ ]:
df_inference_output =  pd.DataFrame(result[0], columns=mp.get_raw_output_columns())

In [ ]:
row_to_plot = df_inference_output.loc[[0]] 

fig = px.scatter(x=row_to_plot["delta_x"], y=row_to_plot["delta_y"])



for i in range(6,12):
    fig.add_scatter(x=row_to_plot[f"out_root_pos_x_{i}"], y=row_to_plot[f"out_root_pos_y_{i}"], mode="markers", marker_symbol="x")

fig.update_traces(marker=dict(size=12,
                              opacity=0.5,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(
    width = 800,
    height = 800,
    title = "fixed-ratio axes"
)
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

In [ ]:
x_columns = df_inference_output.filter(like="out_jpos_x_").values
y_columns = df_inference_output.filter(like="out_jpos_y_").values
z_columns = df_inference_output.filter(like="out_jpos_z_").values

# Flatten the arrays if needed
x_array = x_columns.flatten()
y_array = y_columns.flatten()
z_array = z_columns.flatten()

In [ ]:
inx = df_inference_input.iloc[0].filter(like="jpos_x_").values.flatten()
iny = df_inference_input.iloc[0].filter(like="jpos_y_").values.flatten()
inz = df_inference_input.iloc[0].filter(like="jpos_z_").values.flatten()


In [ ]:
scatter_data = pd.DataFrame({'X': x_array, 'Y': y_array, 'Z': z_array})


max_range = max(scatter_data[['X', 'Y', 'Z']].max())
min_range = min(scatter_data[['X', 'Y', 'Z']].min())

fig = px.scatter_3d(scatter_data, x='X', y='Z', z='Y', title='Out Joint Positions', labels={'X': 'X Values', 'Y': 'Y Values', 'Z': 'Z Values'})
fig.update_traces(name='Out Joint Positions', showlegend = True)
fig.add_scatter3d(x=inx, y=inz, z=iny, mode="markers", name='In Joint Positions')

fig.update_layout(scene=dict(aspectmode="cube", xaxis=dict(range=[min_range, max_range]), yaxis=dict(range=[min_range, max_range]), zaxis=dict(range=[min_range, max_range])))

fig.update_layout(
    width = 800,
    height = 800,
    title = "Joint Position In & Out (Next Frame)"
)
fig.update_traces(marker_size = 3)
fig.show()

In [ ]:
x_columns = df_inference_output.filter(like="out_jrot_0_").values
y_columns = df_inference_output.filter(like="out_jrot_1_").values
z_columns = df_inference_output.filter(like="out_jrot_2_").values

# Flatten the arrays if needed
x_array = x_columns.flatten()
y_array = y_columns.flatten()
z_array = z_columns.flatten()

inx = df_inference_input.iloc[0].filter(like="jrot_0_").values.flatten()
iny = df_inference_input.iloc[0].filter(like="jrot_1_").values.flatten()
inz = df_inference_input.iloc[0].filter(like="jrot_2_").values.flatten()



scatter_data = pd.DataFrame({'X': x_array, 'Y': y_array, 'Z': z_array})


max_range = max(scatter_data[['X', 'Y', 'Z']].max())
min_range = min(scatter_data[['X', 'Y', 'Z']].min())

fig = px.scatter_3d(scatter_data, x='X', y='Z', z='Y', title='Out Joint Positions', labels={'X': 'X Values', 'Y': 'Y Values', 'Z': 'Z Values'})
fig.update_traces(name='Out Joint Positions', showlegend = True)
fig.add_scatter3d(x=inx, y=inz, z=iny, mode="markers", name='In Joint Positions')

#fig.update_layout(scene=dict(aspectmode="cube", xaxis=dict(range=[min_range, max_range]), yaxis=dict(range=[min_range, max_range]), zaxis=dict(range=[min_range, max_range])))

fig.update_layout(
    width = 800,
    height = 800,
    title = "Joint Position In & Out (Next Frame)"
)

fig.update_traces(marker_size = 3)
fig.show()